In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
import collections

conf = SparkConf().setMaster("local").setAppName("RatingHistogram")
sc = SparkContext(conf=conf).setLogLevel()

21/07/10 15:44:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
path_file_movie = r"/opt/project/data/raw/ml-100k/u.data"
lines = sc.textFile(path_file_movie)
# print(lines.collect())
ratings = lines.map(lambda x: x.split()[2])
ratings.countByKey()

defaultdict(int, {'3': 27145, '1': 6110, '2': 11370, '4': 34174, '5': 21201})

In [8]:
lines = sc.textFile(path_file_movie)
ratings = lines.map(lambda x: x.split()[2])
result = ratings.countByValue()

sorted_results = collections.OrderedDict(sorted(result.items()))
for key, value in sorted_results.items():
    print(f"{key}, {value}")



1, 6110
2, 11370
3, 27145
4, 34174
5, 21201


In [8]:
def parse_line(line):
    fileds = line.split(",")
    age = int(fileds[2])
    num_friends = int(fileds[3])
    return (age, num_friends)

path_fake_friends = r"/opt/project/data/raw/fakefriends.csv"
lines = sc.textFile(path_fake_friends)
rdd = lines.map(parse_line)
totals_by_age = rdd.mapValues(lambda x: (x,1))\
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
# print(totals_by_age)
averages_by_age = totals_by_age.mapValues(lambda x: x[0]/x[1]).collect()
print(averages_by_age)

AttributeError: 'NoneType' object has no attribute 'sc'

In [35]:
def parse_line(line):
    fileds = line.split(",")
    station_id = fileds[0]
    label_desc = str(fileds[2])
    value = float(fileds[3]) * 0.1 * (9. / 5.) + 32
    return (station_id, label_desc, value)

path_temperature = r"/opt/project/data/raw/1800.csv"
temps = sc.textFile(path_temperature)
temps = temps.map(parse_line)
temps = temps.filter(lambda x: "TMAX" in x[1])
temps = temps.map(lambda x: (x[0], x[2]))
temps = temps.reduceByKey(lambda x, y: max(x,y))
temps = temps.collect()
print(temps)


[('ITE00100554', 90.14000000000001), ('EZE00100082', 90.14000000000001)]


In [53]:
path_book = r"/opt/project/data/raw/book.txt"
book = sc.textFile(path_book)
words = book.flatMap(lambda x: x.split())
words_count = words.map(lambda x:(x,1)).reduceByKey(lambda x,y: x+y)
words_counts_sorted = words_count.map(lambda x: (x[1],x[0])).sortByKey()
results = words_counts_sorted.collect()

for item in results[::-1]:
    print(item)


(1789, 'to')
(1339, 'your')
(1267, 'you')
(1176, 'the')
(1148, 'a')
(941, 'of')
(901, 'and')
(641, 'that')
(552, 'in')
(531, 'is')
(500, 'for')
(399, 'on')
(391, 'are')
(347, 'be')
(322, 'I')
(319, 'can')
(311, 'it')
(299, 'have')
(297, 'as')
(292, 'with')
(267, 'or')
(261, 'business')
(237, 'If')
(220, 'will')
(208, 'this')
(199, 'my')
(192, 'but')
(192, 'they')
(189, 'at')
(187, 'more')
(181, 'about')
(177, 'what')
(174, 'if')
(174, '�')
(172, 'an')
(169, 'not')
(166, 'need')
(165, 'time')
(161, 'from')
(159, "you're")
(156, 'do')
(155, 'up')
(144, 'You')
(143, 'new')
(138, 'out')
(131, 'just')
(127, 'how')
(125, 'product')
(122, 'people')
(117, 'their')
(116, 'get')
(114, 'work')
(114, 'own')
(111, 'some')
(109, 'all')
(109, 'by')
(108, 'them')
(107, "it's")
(106, 'may')
(101, 'customers')
(101, 'so')
(99, 'want')
(97, "don't")
(92, 'than')
(88, 'make')
(88, 'The')
(85, 'who')
(84, 'also')
(84, 'was')
(83, 'company')
(82, 'there')
(79, 'website')
(78, 'much')
(77, 'only')
(76, 'prob

In [74]:
from typing import Tuple

def pares_line(line:str) -> Tuple[int, float]:
    filed = line.split(",")
    customer_id = int(filed[0])
    price = float(filed[2])
    return (customer_id, price)

path_order = r"/opt/project/data/raw/customer-orders.csv"
customer = sc.textFile(path_order)
customer = customer.map(pares_line)
customer = customer.reduceByKey(lambda x,y: x+y)
customer_sorted = customer.map(lambda x: (x[1],x[0])).sortByKey().collect()
customer_sorted
sc.stop()

AttributeError: 'NoneType' object has no attribute 'sc'

In [10]:
from pyspark.sql import SparkSession, Row

spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

def mapper(line):
    fileds = line.split(",")
    return Row(ID=int(fileds[0]), name=str(fileds[1].encode("utf-8")),
               age=int(fileds[2]), num_firends = int(fileds[3]))

lines = spark.sparkContext.textFile(path_fake_friends)
people = lines.map(mapper)

schema_people = spark.createDataFrame(people).cache()
schema_people.createOrReplaceTempView("people")

teenagers = spark.sql("SELECT * FROM people WHERE age >= 13 AND age <= 19")
print(teenagers.to)
for teen in teenagers.collect():
    print(teen)

schema_people.groupby("age").count().orderBy("age").show(10)

spark.stop()


root
 |-- ID: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- num_firends: long (nullable = true)

None
Row(ID=21, name="b'Miles'", age=19, num_firends=268)
Row(ID=52, name="b'Beverly'", age=19, num_firends=269)
Row(ID=54, name="b'Brunt'", age=19, num_firends=5)
Row(ID=106, name="b'Beverly'", age=18, num_firends=499)
Row(ID=115, name="b'Dukat'", age=18, num_firends=397)
Row(ID=133, name="b'Quark'", age=19, num_firends=265)
Row(ID=136, name="b'Will'", age=19, num_firends=335)
Row(ID=225, name="b'Elim'", age=19, num_firends=106)
Row(ID=304, name="b'Will'", age=19, num_firends=404)
Row(ID=341, name="b'Data'", age=18, num_firends=326)
Row(ID=366, name="b'Keiko'", age=19, num_firends=119)
Row(ID=373, name="b'Quark'", age=19, num_firends=272)
Row(ID=377, name="b'Beverly'", age=18, num_firends=418)
Row(ID=404, name="b'Kasidy'", age=18, num_firends=24)
Row(ID=409, name="b'Nog'", age=19, num_firends=267)
Row(ID=439, name="b'Data'", age=18, num_fi

+---+-----+
|age|count|
+---+-----+
| 18|    8|
| 19|   11|
| 20|    5|
| 21|    8|
| 22|    7|
| 23|   10|
| 24|    5|
| 25|   11|
| 26|   17|
| 27|    8|
+---+-----+
only showing top 10 rows



In [63]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

people = spark.read.option("header", "true").option("inferSchema","true")\
    .csv(path_fake_friends)

people.printSchema()

people.select("name").show()

people.filter(people.age < 21).show()



root
 |-- 0: integer (nullable = true)
 |-- Will: string (nullable = true)
 |-- 33: integer (nullable = true)
 |-- 385: integer (nullable = true)



AnalysisException: cannot resolve '`name`' given input columns: [0, 33, 385, Will];;
'Project ['name]
+- Relation[0#390,Will#391,33#392,385#393] csv


In [9]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func

spark = SparkSession.builder.appName("SparkSql").getOrCreate()

fake_friends_header = r"/opt/project/data/raw/fakefriends-header.csv"

people = spark.read.option("header", "true").option("inferSchema", "true").csv(fake_friends_header)

people.printSchema()

friends = people.select("age", "friends")

friends.groupBy("age").avg("friends").sort("age").show()


friends.groupBy("age").agg(func.round(func.avg("friends"),2 ).alias("firends_avg")).sort("age").show()

root
 |-- userID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends: integer (nullable = true)



+---+------------------+
|age|      avg(friends)|
+---+------------------+
| 18|           343.375|
| 19|213.27272727272728|
| 20|             165.0|
| 21|           350.875|
| 22|206.42857142857142|
| 23|             246.3|
| 24|             233.8|
| 25|197.45454545454547|
| 26|242.05882352941177|
| 27|           228.125|
| 28|             209.1|
| 29|215.91666666666666|
| 30| 235.8181818181818|
| 31|            267.25|
| 32| 207.9090909090909|
| 33| 325.3333333333333|
| 34|             245.5|
| 35|           211.625|
| 36|             246.6|
| 37|249.33333333333334|
+---+------------------+
only showing top 20 rows



+---+-----------+
|age|firends_avg|
+---+-----------+
| 18|     343.38|
| 19|     213.27|
| 20|      165.0|
| 21|     350.88|
| 22|     206.43|
| 23|      246.3|
| 24|      233.8|
| 25|     197.45|
| 26|     242.06|
| 27|     228.13|
| 28|      209.1|
| 29|     215.92|
| 30|     235.82|
| 31|     267.25|
| 32|     207.91|
| 33|     325.33|
| 34|      245.5|
| 35|     211.63|
| 36|      246.6|
| 37|     249.33|
+---+-----------+
only showing top 20 rows



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func

spark = SparkSession.builder.appName("SaprkSQL").getOrCreate()

inputDF = spark.read.text(path_book)

In [80]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructField, StructType, StringType, FloatType, IntegerType

spark = SparkSession.builder.appName("XXX").getOrCreate()

schema = StructType([
    StructField("ID", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("price", FloatType(), False),
])

df = spark.read.schema(schema).csv(path_order)

df = df.select("ID","price").groupBy("ID").agg(func.round(func.sum("price"),2).alias("summed_price"))
df = df.sort("summed_price", ascending=False)
df.show()


spark.stop

+---+------------+
| ID|summed_price|
+---+------------+
| 68|     6375.45|
| 73|      6206.2|
| 39|     6193.11|
| 54|     6065.39|
| 71|     5995.66|
|  2|     5994.59|
| 97|     5977.19|
| 46|     5963.11|
| 42|     5696.84|
| 59|     5642.89|
| 41|     5637.62|
|  0|     5524.95|
|  8|     5517.24|
| 85|     5503.43|
| 61|     5497.48|
| 32|     5496.05|
| 58|     5437.73|
| 63|     5415.15|
| 15|     5413.51|
|  6|     5397.88|
+---+------------+
only showing top 20 rows



<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f7678789100>>

In [6]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import  functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

import codecs

path_movie_item

def load_movie_names():
    movie_name = {}

    with codecs.open()

spark = SparkSession.builder.appName("X").getOrCreate()

schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("movie_id", IntegerType(), True),
    StructField("rating", IntegerType(), True),
    StructField("timestamp", IntegerType(), True),
])

movies_df = spark.read.option("sep", "\t").schema(schema).csv(path_file_movie)

top_movie_ids = movies_df.groupBy("movie_id").count()



def look_up_name(movie_id):
    return nameD

top_movie_ids.show(10)
spark.stop()


+--------+-----+
|movie_id|count|
+--------+-----+
|      50|  583|
|     258|  509|
|     100|  508|
|     181|  507|
|     294|  485|
|     286|  481|
|     288|  478|
|       1|  452|
|     300|  431|
|     121|  429|
+--------+-----+
only showing top 10 rows



In [26]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, FloatType

PATH_MARVEL_GRAPH = r"/opt/project/data/raw/Marvel-graph.txt"
PATH_MARVEL_NAME = r"/opt/project/data/raw/Marvel-names.txt"

spark = SparkSession.builder.appName("X").getOrCreate()

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
])

names = spark.read.schema(schema).option("sep", " ").csv(PATH_MARVEL_NAME)

lines = spark.read.text(PATH_MARVEL_GRAPH)

connections = lines.withColumn("id", func.split(func.col("value"), " ")[0])\
    .withColumn("connections", func.size(func.split(func.col("value"), " ")) - 1)\
    .groupBy("id").agg(func.sum("connections").alias("connections"))

most_popular = connections.sort(func.col("connections").desc()).first()

most_popular_name = names.filter(func.col("id") == 5736).select("name").first()

most_popular = connections.sort(func.col("connections").desc()).show(10)

most_popular_name


+----+-----------+
|  id|connections|
+----+-----------+
| 859|       1937|
|5306|       1745|
|2664|       1532|
|5716|       1429|
|6306|       1397|
|3805|       1389|
|2557|       1374|
|4898|       1348|
|5736|       1292|
| 403|       1283|
+----+-----------+
only showing top 10 rows



Row(name='THOR/DR. DONALD BLAK')